In [57]:
import numpy as np
import pandas as pd

In [60]:
df_credits = pd.read_csv('datasets/credits.csv')
df_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [61]:
df_meta = pd.read_csv('datasets/movies_metadata.csv')
df_meta.head()

/tmp/ipykernel_988/2973920076.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta = pd.read_csv('datasets/movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [62]:
df_meta['release_date'] = pd.to_datetime(df_meta['release_date'], errors='coerce')

In [63]:

df_meta['year'] = df_meta['release_date'].dt.year

In [64]:
df_meta['year'].value_counts().sort_index()

year
1874.0       1
1878.0       1
1883.0       1
1887.0       1
1888.0       2
          ... 
2015.0    1905
2016.0    1604
2017.0     532
2018.0       5
2020.0       1
Name: count, Length: 135, dtype: int64

In [66]:
df_meta_2017 = df_meta.loc[df_meta.year <= 2017,['genres','id','title','year']]

In [67]:
df_meta_2017.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45370 entries, 0 to 45465
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   genres  45370 non-null  object 
 1   id      45370 non-null  object 
 2   title   45370 non-null  object 
 3   year    45370 non-null  float64
dtypes: float64(1), object(3)
memory usage: 1.7+ MB


In [68]:
df_meta_2017['id'] = df_meta_2017['id'].astype(int)

In [81]:
data = pd.merge(df_meta_2017, df_credits, on='id')

In [82]:
# evaluates an expression node or a string containing a Python literal or container display
import ast
data['genres'] = data['genres'].map(lambda x: ast.literal_eval(x))
data['cast'] = data['cast'].map(lambda x: ast.literal_eval(x))
data['crew'] = data['crew'].map(lambda x: ast.literal_eval(x))

In [83]:
def make_genresList(x):
    gen = []
    for item in x:
        if item['name'] == 'Science Fiction':
            gen.append('Sci-Fi')
        else:
            gen.append(item['name'])
    if gen == []:
        return np.NaN
    else:
        return (" ".join(gen))

data['genres_list'] = data['genres'].map(lambda x: make_genresList(x))

In [84]:
def get_actors(x, n):
    if len(x) >= n:
        return x[n - 1]  # Subtract 1 as indexing starts from 0
    else:
        return np.NaN

data['casts'] = data['cast'].map(lambda x: [i.get('name') for i in x])
data['actor_1_name'] = data['casts'].map(lambda x: get_actors(x, 1))
data['actor_2_name'] = data['casts'].map(lambda x: get_actors(x, 2))
data['actor_3_name'] = data['casts'].map(lambda x: get_actors(x, 3))

In [85]:
def get_directors(x):
    dt = []
    for item in x:
        if item['job'] == 'Director':
            dt.append(item['name'])
    if dt == []:
        return np.NaN
    else:
        return (" ".join(dt))
data['director_name'] = data['crew'].map(lambda x: get_directors(x))

In [86]:
data.drop(['id', 'genres', 'cast', 'casts','crew','year'], axis=1, inplace=True)
data.head()

,title,genres_list,actor_1_name,actor_2_name,actor_3_name,director_name
0,Toy Story,Animation Comedy Family,Tom Hanks,Tim Allen,Don Rickles,John Lasseter
1,Jumanji,Adventure Fantasy Family,Robin Williams,Jonathan Hyde,Kirsten Dunst,Joe Johnston
2,Grumpier Old Men,Romance Comedy,Walter Matthau,Jack Lemmon,Ann-Margret,Howard Deutch
3,Waiting to Exhale,Comedy Drama Romance,Whitney Houston,Angela Bassett,Loretta Devine,Forest Whitaker
4,Father of the Bride Part II,Comedy,Steve Martin,Diane Keaton,Martin Short,Charles Shyer


In [87]:
data = data.rename(columns={'genres_list':'genres'})
data = data.rename(columns={'title':'movie_title'})

In [88]:
data = data.dropna(how='any')

In [89]:
data['movie_title'] = data['movie_title'].str.lower()

In [90]:
data['comb'] = data['actor_1_name'] + ' ' + data['actor_2_name'] + ' '+ data['actor_3_name'] + ' '+ data['director_name'] +' ' + data['genres']

In [91]:
data.head()

,movie_title,genres,actor_1_name,actor_2_name,actor_3_name,director_name,comb
0,toy story,Animation Comedy Family,Tom Hanks,Tim Allen,Don Rickles,John Lasseter,Tom Hanks Tim Allen Don Rickles John Lasseter ...
1,jumanji,Adventure Fantasy Family,Robin Williams,Jonathan Hyde,Kirsten Dunst,Joe Johnston,Robin Williams Jonathan Hyde Kirsten Dunst Joe...
2,grumpier old men,Romance Comedy,Walter Matthau,Jack Lemmon,Ann-Margret,Howard Deutch,Walter Matthau Jack Lemmon Ann-Margret Howard ...
3,waiting to exhale,Comedy Drama Romance,Whitney Houston,Angela Bassett,Loretta Devine,Forest Whitaker,Whitney Houston Angela Bassett Loretta Devine ...
4,father of the bride part ii,Comedy,Steve Martin,Diane Keaton,Martin Short,Charles Shyer,Steve Martin Diane Keaton Martin Short Charles...


In [92]:
data.drop_duplicates(subset ="movie_title", keep = 'last', inplace = True)

In [85]:
data.to_csv('movie.csv',index=False)